This Python script employs a five-folds leveraging the `mp_time_split` subpackage. By treating materials discovered in the future as generated data, the script benchmarks the progress of materials discovery over time. Each fold partitions the dataset into distinct training, validation, and generated sets, simulating the incremental advancement of materials science.

<a href="https://colab.research.google.com/github/sparks-baird/matbench-genmetrics/blob/main/notebooks/core/2.0-matbench-genmetrics-materials_discovery_progress_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="red">**NOTE: If using Colab, "Restart Runtime" after installation.**</font>

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
try:
    import google.colab  # type: ignore # noqa: F401
    %pip install git+https://github.com/sparks-baird/matbench-genmetrics.git
except ImportError:
    print("not in Colab")

In [1]:
from matbench_genmetrics.core.metrics import MPTSMetrics
from tqdm.notebook import tqdm
from pprint import pprint

In [3]:
%%time
n = 5396
mptm = MPTSMetrics(dummy=False, verbose=True, num_gen=n)
mptm2 = MPTSMetrics(dummy=False, verbose=True, num_gen=n)
for fold in tqdm(mptm.folds[0:4]):  
    train_val_inputs_1, test_structures = mptm.get_train_and_val_data(fold, return_test=True)
    train_val_inputs_2, gen_structures = mptm2.get_train_and_val_data(fold + 1, return_test=True)

    mptm.evaluate_and_record(fold, gen_structures)
    
pprint(mptm.recorded_metrics)

  0%|          | 0/4 [00:00<?, ?it/s]

Reading file c:\Users\hasan\miniconda3\envs\matbench-genmetrics\lib\site-packages\matbench_genmetrics\mp_time_split\utils\mp_time_summary.json.gz: 0it [00:47, ?it/s]#####9| 40453/40476 [00:46<00:00, 874.82it/s] 
Decoding objects from c:\Users\hasan\miniconda3\envs\matbench-genmetrics\lib\site-packages\matbench_genmetrics\mp_time_split\utils\mp_time_summary.json.gz: 100%|##########| 40476/40476 [00:47<00:00, 860.28it/s]
Reading file c:\Users\hasan\miniconda3\envs\matbench-genmetrics\lib\site-packages\matbench_genmetrics\mp_time_split\utils\mp_time_summary.json.gz: 0it [00:54, ?it/s]#####9| 40460/40476 [00:53<00:00, 624.34it/s] 
Decoding objects from c:\Users\hasan\miniconda3\envs\matbench-genmetrics\lib\site-packages\matbench_genmetrics\mp_time_split\utils\mp_time_summary.json.gz: 100%|##########| 40476/40476 [00:54<00:00, 747.36it/s]
SiteStatsFingerprint: 100%|██████████| 5396/5396 [42:20<00:00,  2.12it/s]
Reading file c:\Users\hasan\miniconda3\envs\matbench-genmetrics\lib\site-package

{0: {'coverage': 0.03280207561156412,
     'novelty': 0.9571905114899926,
     'uniqueness': 0.9998874668429091,
     'validity': 0.9657295385947532},
 1: {'coverage': 0.032987398072646404,
     'novelty': 0.948295033358043,
     'uniqueness': 0.9999187947547732,
     'validity': 0.9689630506461338},
 2: {'coverage': 0.019644180874722018,
     'novelty': 0.9529280948851001,
     'uniqueness': 0.999984404745629,
     'validity': 0.942820170335924},
 3: {'coverage': 0.02575982209043736,
     'novelty': 0.939770200148258,
     'uniqueness': 0.9999801452488405,
     'validity': 0.8635418248589397}}
CPU times: total: 4min 30s
Wall time: 3h 48min 31s
